In [15]:
import cv2
import numpy as np
from pdf2image import convert_from_path
import sys, fitz  # import the bindings


In [19]:
# Store Pdf with convert_from_path function
path = 'pdfs_google_patent/1498383555865983980-08507653 - US8507653.pdf'

#1498392494248982643-08003742 - US8003742.pdf

In [20]:
doc = fitz.open(path)  # open document
for page in doc:  # iterate through the pages
    pix = page.get_pixmap()  # render page to an image
    pix.save("testpull_ocv_pages/page-%i.png" % page.number)  # store image as a PNG


In [23]:
page_path = 'testpull_ocv_pages/page-10.png'

# Load image, grayscale, Otsu's threshold
image = cv2.imread(page_path)
original = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Dilate with horizontal kernel
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20,10))
dilate = cv2.dilate(thresh, kernel, iterations=2)

# Find contours and remove non-diagram contours
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    area = cv2.contourArea(c)
    if w/h > 2 and area > 10000:
        cv2.drawContours(dilate, [c], -1, (0,0,0), -1)

# Iterate through diagram contours and form single bounding box
boxes = []
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    boxes.append([x,y, x+w,y+h])

boxes = np.asarray(boxes)
x = np.min(boxes[:,0])
y = np.min(boxes[:,1])
w = np.max(boxes[:,2]) - x
h = np.max(boxes[:,3]) - y

# Extract ROI
cv2.rectangle(image, (x,y), (x + w,y + h), (36,255,12), 3)
ROI = original[y:y+h, x:x+w]

cv2.imshow('image', image)
cv2.imshow('thresh', thresh) 
cv2.imshow('dilate', dilate)
cv2.imshow('ROI', ROI)
cv2.waitKey(0)

-1